### Q1

##### To run elastic search

```
docker run -d \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    -v ./elastic_data:/usr/share/elasticsearch/data \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [1]:
import requests

In [2]:
res = requests.get("http://localhost:9200")

In [3]:
res.json()["version"]["build_hash"]

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2

In [5]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/mnt/c/Users/otaci/Desktop/repos/llm-zoomcamp-otacilio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
es_client = Elasticsearch('http://localhost:9200') 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:06<00:00, 150.91it/s]


### Q3

In [59]:
def elastic_search(query, size=5, query_filter=False):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    if query_filter:
        search_query['query']['bool'].update(query_filter)

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        hit['_source']['_score'] = hit['_score']
        result_docs.append(hit['_source'])
    
    return result_docs

In [60]:
query = "How do I execute a command in a running docker container?"

In [61]:
es_res = elastic_search(query)

In [67]:
es_res[0]["_score"]

84.050095

### Q4

In [63]:
query_filter = {
    "filter": {
        "term": {
             "course": "machine-learning-zoomcamp"
            }
        }
    }

In [68]:
q4_results = elastic_search(query, size=3, query_filter=query_filter)
q4_results[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5

In [71]:
context = ""
context_template = """
Q: {question}
A: {text}
""".strip()

for r in q4_results:
    context += context_template.format(question=r["question"], text=r["text"]) + "\n\n"

In [73]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

query = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=query, context=context)

print(len(prompt))

1464


### Q6

In [76]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [77]:
prompt_encoded = encoding.encode(prompt)

In [79]:
print(len(prompt_encoded))

323
